In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# read in all the words
words = open('names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [4]:
len(words)

32033

In [5]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0 # add the special 'start/end' character
itos = {i:s for s,i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [45]:
# Build the dataset

block_size = 3 # context length: how many characters do we take to predict the next one?

X, Y = [], [] # X is the input, Y is the target

for w in words:
    context = [0] * block_size
    for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        context = context[1:] + [ix]

X = torch.tensor(X)
Y = torch.tensor(Y)


In [7]:
C = torch.randn((27, 2))

In [19]:
emb = C[X] # embed each character in the context
emb.shape

torch.Size([32, 3, 2])

In [18]:
W1 = torch.randn((6, 100))
b1 = torch.randn(100)

In [27]:
h = torch.tanh(emb.view(-1, 6) @ W1 + b1)

In [28]:
h

tensor([[ 0.9379,  0.9268, -0.2867,  ..., -0.9999,  0.8383,  0.0363],
        [ 0.9380,  0.8670, -0.4084,  ..., -0.9999,  0.8842,  0.1667],
        [ 0.9736,  0.9829,  0.5789,  ..., -0.9992,  0.6240,  0.6242],
        ...,
        [ 0.9990,  0.9900, -0.6706,  ..., -0.9970, -0.3945, -0.9111],
        [-0.9975, -0.9839, -0.5745,  ..., -0.8051,  0.8815,  0.6220],
        [ 0.9897, -0.3182, -0.9871,  ..., -0.9975,  0.7738, -0.8172]])

In [24]:
h.shape

torch.Size([32, 100])

In [29]:
W2 = torch.randn((100, 27))
b2 = torch.randn(27)

In [30]:
logits = h @ W2 + b2   

In [31]:
logits.shape

torch.Size([32, 27])

In [32]:
counts = logits.exp()

In [33]:
prob = counts / counts.sum(1, keepdim=True)

In [34]:
prob.shape

torch.Size([32, 27])

# ----------------- Now made respectable --------------------

In [46]:
X.shape, Y.shape

(torch.Size([228146, 3]), torch.Size([228146]))

In [47]:
g = torch.Generator().manual_seed(2147483647) # for reproducibility
C = torch.randn((27, 2), generator=g)
W1 = torch.randn((6, 100), generator=g)
b1 = torch.randn(100, generator=g)
W2 = torch.randn((100, 27), generator=g)
b2 = torch.randn(27, generator=g)
parameters = [C, W1, b1, W2, b2]

In [48]:
sum(p.numel() for p in parameters) # number of parameters in total

3481

In [49]:
for p in parameters:
    p.requires_grad = True

In [50]:
for _ in range(10):
    # forward pass
    emb = C[X] # embed each character in the context (32, 3, 2)
    h = torch.tanh(emb.view(-1, 6) @ W1 + b1) # hidden layer (32, 100)
    logits = h @ W2 + b2 # (32, 27)
    loss = F.cross_entropy(logits, Y)
    print(loss.item())
    # backward pass
    for p in parameters:
        p.grad = None
    loss.backward()
    # update parameters
    for p in parameters:
        p.data += -0.1 * p.grad

19.505229949951172
17.08448600769043
15.776532173156738
14.833342552185059
14.002608299255371
13.253263473510742
12.57991886138916
11.983102798461914
11.470495223999023
11.05185604095459


tensor(17.7697)